# Application

In [1]:
import os
import pandas as pd
import numpy as np

from glob import glob
from scipy.spatial.distance import cdist
from scipy.linalg import solve
from scipy.optimize import curve_fit
# from geopy.distance import geodesic
from pykrige.ok import OrdinaryKriging


## def

In [5]:

# calculate_semivariogram 计算半方差函数
def calculate_semivariogram(data):
    num_points = len(data)
    semivariances = []

    for i in range(num_points):
        for j in range(i + 1, num_points):
            dist = np.linalg.norm([data['Latitude'].iloc[i] - data['Latitude'].iloc[j],
                                   data['Longitude'].iloc[i] - data['Longitude'].iloc[j]])
            squared_diff = (data['Value'].iloc[i] - data['Value'].iloc[j]) ** 2
            semivariances.append((dist, squared_diff))

    unique_distances = sorted(set([item[0] for item in semivariances]))
    avg_semivariances = []
    for dist in unique_distances:
        squared_diffs = [item[1] for item in semivariances if item[0] == dist]
        avg_semivariances.append((dist, np.mean(squared_diffs) / 2.0))

    return np.array(avg_semivariances)

# calculate_kriging_weights 计算克里金权重
def calculate_kriging_weights(semivariogram, distances, n, nugget=1e-10):
    A = np.zeros((n + 1, n + 1))
    
    for i in range(n):
        for j in range(n):
            if i == j:
                A[i, j] = semivariogram[0][1] + nugget
            else:
                dist = int(distances[0, j])
                A[i, j] = semivariogram[dist][1] if dist < len(semivariogram) else semivariogram[-1][1]

    A[-1, :-1] = 1
    A[:-1, -1] = 1

    b = np.zeros(n + 1)
    for i in range(n):
        dist = int(distances[0, i])
        b[i] = semivariogram[dist][1] if dist < len(semivariogram) else semivariogram[-1][1]

    weights = solve(A, b)
    return weights[:-1]


def adjust_weights(weights, wind_speed, wind_dir, sensor_directions, max_wind_speed):
    adjustments = 1 + (wind_speed * np.cos(np.radians(wind_dir - sensor_directions))) / max_wind_speed
    adjusted_weights = np.clip(weights * adjustments, 0, None)  # Ensure weights are non-negative
    return adjusted_weights

def normalize_weights(weights):
    total_weight = np.sum(weights)
    if total_weight == 0:
        return np.zeros_like(weights)
    return weights / total_weight



In [6]:

# interpolate 插值估算
def interpolate(data, weights):
    return np.sum(weights * data['Value'].values)


# 修改 get_wind_data 函数
def get_wind_data(date, year):
    wind_data_path = f'{meteostat_folder}\meteostat{year}.csv'
    wind_data_df = pd.read_csv(wind_data_path, encoding='utf-8-sig')  # 处理BOM
    wind_data_df.columns = wind_data_df.columns.str.strip()

    # # Debugging: Print column names to verify
    # print("Column names in wind_data_df:", wind_data_df.columns)

    # 确保日期格式一致
    wind_data_df['date'] = pd.to_datetime(wind_data_df['date'], format='%Y-%m-%d').dt.date
    date = pd.to_datetime(date, format='%Y-%m-%d').date()  # 确保date参数也被转换成相同格式

    wind_info = wind_data_df[wind_data_df['date'] == date]
    if not wind_info.empty:
        return wind_info['wspd'].values[0], wind_info['wdir'].values[0]
    else:
        return np.nan, np.nan


def interpolate_and_clip(df, column, method='linear', order=2):
    # 获取原始列的最小值和最大值
    original_min = df[column].min()
    original_max = df[column].max()
    
    # 进行插值
    if method == 'polynomial':
        interpolated_values = df[column].interpolate(method=method, order=order)
    else:
        interpolated_values = df[column].interpolate(method=method)
    
    # 确保插值后的值在原来值的 {min, max} 范围之间
    interpolated_values = interpolated_values.clip(lower=original_min, upper=original_max).round(2)
    
    # 仅更新空值部分
    return df[column].combine_first(interpolated_values)

# 自定义的多项式插值函数
def polynomial_interpolation(series, order=2):
    return series.interpolate(method='polynomial', order=order)

<>:8: SyntaxWarning: invalid escape sequence '\m'
<>:8: SyntaxWarning: invalid escape sequence '\m'
C:\Users\Chailee\AppData\Local\Temp\ipykernel_20936\1101129135.py:8: SyntaxWarning: invalid escape sequence '\m'
  wind_data_path = f'{meteostat_folder}\meteostat{year}.csv'


In [9]:
def preprocess_data(base_folder, region):
    # 构建文件路径
    region_folder = os.path.join(base_folder, region)
    no_path = os.path.join(region_folder, f'{region}-Nitrogen Dioxide (ug m-3).csv')
    coords_path = os.path.join(base_folder, 'coords_londonair.csv')
    
    # 读取数据
    no_df = pd.read_csv(no_path)
    coords_df = pd.read_csv(coords_path)
    
    # 数据预处理
    no_df['ReadingDateTime'] = pd.to_datetime(no_df['ReadingDateTime'], format='%d/%m/%Y %H:%M')
    no_df['Date'] = no_df['ReadingDateTime'].dt.date
    coords_df[['Latitude', 'Longitude']] = coords_df['Latitude & Longitude'].str.split(', ', expand=True).astype(float)
    
    # 合并坐标信息
    no_df = pd.merge(no_df, coords_df[['Site', 'Latitude', 'Longitude']], on='Site', how='left')
    
    # 只处理排在最前面的第一种“Site”的Value值
    first_site = no_df['Site'].unique()[0]
    no_df = no_df[no_df['Site'] == first_site]
    
    # 检查并处理 NaN 值
    if no_df.isnull().values.any():
        print(f"NaN values found in {region} data before interpolation")
    
    # 对于第一个site的任何空缺时间的值，根据其他年份同日期的数据规律进行填充
    no_df.set_index('ReadingDateTime', inplace=True)
    monthly_daily_mean = no_df.groupby([no_df.index.month, no_df.index.day])['Value'].transform('mean')
    no_df['Value'].fillna(monthly_daily_mean, inplace=True)
    no_df.reset_index(inplace=True)
    
    # 检查插值结果
    if no_df['Value'].isnull().values.any():
        print(f"NaN values found in {region} data after interpolation")
    
    return no_df

In [29]:
# Breathe London data preprocessing
def preprocess_data(base_folder, breathe_folder, region):
    # 构建文件路径
    region_folder = os.path.join(base_folder, region)
    no_path = os.path.join(region_folder, f'{region}-Nitrogen Dioxide (ug m-3).csv')
    coords_path = os.path.join(base_folder, 'coords_londonair.csv')
    
    # 读取数据
    no_df = pd.read_csv(no_path)
    coords_df = pd.read_csv(coords_path)
    
    # 数据预处理
    no_df['ReadingDateTime'] = pd.to_datetime(no_df['ReadingDateTime'], format='%d/%m/%Y %H:%M')
    no_df['Date'] = no_df['ReadingDateTime'].dt.date
    coords_df[['Latitude', 'Longitude']] = coords_df['Latitude & Longitude'].str.split(', ', expand=True).astype(float)
    
    # 合并坐标信息
    no_df = pd.merge(no_df, coords_df[['Site', 'Latitude', 'Longitude']], on='Site', how='left')
    
    # 只处理排在最前面的第一种“Site”的Value值
    first_site = no_df['Site'].unique()[0]
    no_df = no_df[no_df['Site'] == first_site]
    
    # 检查并处理 NaN 值
    if no_df.isnull().values.any():
        print(f"NaN values found in {region} data before interpolation")
    
    # 对于第一个site的任何空缺时间的值，根据其他年份同日期的数据规律进行填充
    no_df.set_index('ReadingDateTime', inplace=True)
    monthly_daily_mean = no_df.groupby([no_df.index.month, no_df.index.day])['Value'].transform('mean')
    no_df['Value'].fillna(monthly_daily_mean, inplace=True)
    no_df.reset_index(inplace=True)
    
    # 如果 2021 年之后的数据有缺失，则使用数据库 BreatheLondon2 对对应地区的 NO2 记录进行补充
    if no_df[no_df['ReadingDateTime'] > '27/01/2021']['Value'].isnull().values.any():
        breathe_region_folder = os.path.join(breathe_folder, region)
        breathe_files = [f for f in os.listdir(breathe_region_folder) if f.endswith('_NO2.csv')]
        
        for file in breathe_files:
            try:
                breathe_no2_df = pd.read_csv(os.path.join(breathe_region_folder, file))
                
                # 统一日期格式
                breathe_no2_df['Date'] = pd.to_datetime(breathe_no2_df['Category'], format='%Y-%m-%d')
                breathe_no2_df.rename(columns={'NO2_mean': 'Value'}, inplace=True)
                
                # 合并数据
                no_df = pd.merge(no_df, breathe_no2_df[['ReadingDateTime', 'Value']], on='ReadingDateTime', how='left', suffixes=('', '_breathe'))
                no_df['Value'] = no_df['Value'].combine_first(no_df['Value_breathe'])
                no_df.drop(columns=['Value_breathe'], inplace=True)
            
            except ValueError as e:
                print(f"Error processing file {file}: {e}")
            except Exception as e:
                print(f"Unexpected error processing file {file}: {e}")
    # 检查插值结果
    if no_df['Value'].isnull().values.any():
        print(f"NaN values found in {region} data after interpolation")
    
    return no_df

## 7 region

In [24]:
# 定义地区列表
regions = [
    'Camden', 'City of London', 'Islington', 
    'Kensington and Chelsea', 'Lambeth', 
    'Southwark', 'Westminster'
]

# 定义文件夹路径
base_folder = r'D:\File_auto\0_UCL_CASA\OneDrive - University College London\Xiaoyi_dissertation\Analysis\Data\AirQuality\LondonAir'
meteostat_folder = r'D:\File_auto\0_UCL_CASA\OneDrive - University College London\Xiaoyi_dissertation\Analysis\Data\meteostat\\'
NO_weighted_output_folder = r'D:\File_auto\0_UCL_CASA\OneDrive - University College London\Xiaoyi_dissertation\Analysis\Data_output\NO_weighted\\'

breathe_folder = 'D:\File_auto\0_UCL_CASA\OneDrive - University College London\Xiaoyi_dissertation\Analysis\Data\\AirQuality\\BreatheLondon2'

# 确保输出文件夹存在
if not os.path.exists(NO_weighted_output_folder):
    os.makedirs(NO_weighted_output_folder)


<>:13: SyntaxWarning: invalid escape sequence '\F'
<>:13: SyntaxWarning: invalid escape sequence '\F'
C:\Users\Chailee\AppData\Local\Temp\ipykernel_20936\1358017423.py:13: SyntaxWarning: invalid escape sequence '\F'
  breathe_folder = 'D:\File_auto\0_UCL_CASA\OneDrive - University College London\Xiaoyi_dissertation\Analysis\Data\\AirQuality\\BreatheLondon2'


In [26]:

# 遍历每个地区
for region in regions:
    print(f"Processing region: {region}")
    
    no_df = preprocess_data(base_folder, region)
    
    # Kriging 计算
    weighted_values = []
    dates = no_df['Date'].unique()
    
    for date in dates:
        daily_data = no_df[no_df['Date'] == date]
        if len(daily_data) > 1:
            semivariogram = calculate_semivariogram(daily_data)
            distances = cdist(daily_data[['Latitude', 'Longitude']], daily_data[['Latitude', 'Longitude']], metric='euclidean')
            
            # 检查并处理 NaN 和 inf 值
            if np.isnan(distances).any() or np.isinf(distances).any():
                print(f"NaN or inf values found in distances for {region} on {date}")
                distances = np.nan_to_num(distances, nan=0.0, posinf=0.0, neginf=0.0)
            
            kriging_weights = calculate_kriging_weights(semivariogram, distances, len(daily_data))
            
            wind_speed, wind_dir = get_wind_data(date, date.year)
            sensor_directions = np.arctan2(daily_data['Longitude'] - daily_data['Longitude'].mean(), daily_data['Latitude'] - daily_data['Latitude'].mean()) * 180 / np.pi
            meteostat_path = f'{meteostat_folder}\\meteostat{date.year}.csv'
            meteostat_df = pd.read_csv(meteostat_path)
            max_wind_speed = meteostat_df['wspd'].max()
            
            adjusted_weights = adjust_weights(kriging_weights, wind_speed, wind_dir, sensor_directions, max_wind_speed)
            normalized_weights = normalize_weights(adjusted_weights)
            
            weighted_value = interpolate(daily_data, normalized_weights)
            weighted_value = max(0, weighted_value)  # 确保没有负值
            weighted_values.append({'Date': date, 'NO2_weighted_value(ug m-3)': weighted_value})
        else:
            weighted_values.append({'Date': date, 'NO2_weighted_value(ug m-3)': daily_data['Value'].values[0]})
    
    # 保存加权后的值
    weighted_df = pd.DataFrame(weighted_values)
    output_path = os.path.join(NO_weighted_output_folder, f'{region}-NO2_weighted.csv')
    weighted_df.to_csv(output_path, index=False)
    print(f"Weighted data saved to {output_path}")

Processing region: Camden


ValueError: embedded null character